In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import xgboost_train as xgb


In [4]:
import torch

# Load embeddings
mutated_embed_dict = torch.load("esm2_embeddings.pt")  # mutated
delta_embed_dict = torch.load("esm2_delta_embeddings.pt")  # wild - mutated delta


In [5]:
# Prepare clinical features
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
import sys
sys.path.append("Utils")  # or "../Utils" if notebook is outside
from tools import stage_mapping, clean_grade

df = stage_mapping(df)
df["Tumour Grade"] = df["Tumour Grade"].apply(clean_grade)

# Filter to ensure we only use samples with available embeddings
df = df[df["mutated_protein"].isin(mutated_embed_dict)]

# Clinical processing
categorical_cols = ["Donor Sex", "Tumour Grade"]
numeric_cols = ["Donor Age at Diagnosis"]

ct = ColumnTransformer([
    ("cat", OneHotEncoder(sparse_output=False, handle_unknown="ignore"), categorical_cols),
    ("num", StandardScaler(), numeric_cols)
])

X_clinical = ct.fit_transform(df[categorical_cols + numeric_cols])

# Create embedding matrix from matched rows
X_embed = np.vstack([mutated_embed_dict[pid].numpy() for pid in df["mutated_protein"]])
# Or optionally: X_embed = np.vstack([delta_embed_dict[pid].numpy() for pid in df["mutated_protein"]])

# Combine features
X = np.hstack([X_clinical, X_embed])
y = LabelEncoder().fit_transform(df["Mapped Cancer Stage"])


NameError: name 'df' is not defined

In [ ]:
from sklearn.model_selection import train_test_split
import xgboost_train as xgb
from sklearn.metrics import classification_report

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

model = xgb.XGBClassifier(use_label_encoder=False, eval_metric="mlogloss")
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))
